# Table of Contents
* [Exploring fenaroli parsing](#Exploring-fenaroli-parsing)
	* [Loading plaintext generated by slate (on AWS instance)](#Loading-plaintext-generated-by-slate-%28on-AWS-instance%29)
	* [RE for flavor compounds](#RE-for-flavor-compounds)
	* [RE for ingredients](#RE-for-ingredients)


In [2]:
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2

In [3]:
import cPickle as pickle
import slate 
import re

In [6]:
import matplotlib.pylab as plt
import matplotlib as mpl
plt.rcParams['figure.figsize'] = (12.0, 6.0)
%matplotlib inline
%load_ext base16_mplrc
%base16_mplrc light default

# Exploring fenaroli parsing

## Loading plaintext generated by slate (on AWS instance)

In [296]:
with open('../data/doc_joined.pkl', 'r') as f:
    joined = pickle.load(f)

In [295]:
cleaned_joined = joined.replace('.', ' ')

In [542]:
# cleaned_joined[:int(len(cleaned_joined)*0.2)]

In [380]:
# cleaned_joined[int(len(cleaned_joined)*0.6):int(len(cleaned_joined)*0.7)]

In [375]:
with open ('joined_inspect', 'w') as f:
    f.write(cleaned_joined)

entries I'm interested in start at split doc 17

## RE for flavor compounds and ingredients

This is the ultimate regex pattern. It finds the natural occurence, and searches below for the next flavor compound
'Natural occurrence: (.+)\n{2,5}[\x0c]?(.*[A-Z]{2,20}.*)\n+'

In [649]:
# sq = re.compile('Natural occurrence:(.+)\n\n[\x0c]?(.*?[A-Z]{2,20}.*?)\n\n(.*?[A-Z]{2,20}.*?)?\n?\n?') defunct for now 
sq = re.compile('Natural occurrence:(.+)\n\n[\x0c]?(.*?[A-Z]{2,20}.*?)\n\n')
raw_ingredients = re.findall(sq, cleaned_joined)

ingredient_series = pd.Series([i[1] for i in raw_ingredients])

In [650]:
len(raw_ingredients)

In [545]:
stop_strings = ingredient_series.value_counts()[:10].index[:4]

top_terms = list(ingredient_series.value_counts().index.values[:10])
stop_strings=top_terms[:4]+ top_terms[5:7] + [top_terms[8]]

# cleaned_ingredients = [comp.strip() for comp in raw_ingredients if comp not in stop_strings]
# cleaned_ingredients[:10]

with_three = [i for i in ingreient_tuple_list if i[2]]

In [638]:
ingreient_tuple_list = []
for i in raw_ingredients:
    if i[2][:7] == 'Synonym':
        ingreient_tuple_list.append((i[0],i[1],''))
    else:
        ingreient_tuple_list.append(i)                 

## Here I have the material I need to build the ingredient:compound mapping

In [615]:
for i in raw_ingredients:
    print i[0], '\n', i[1], '\n', i[2]

In [646]:
ingreient_tuple_list

In [663]:
flavor_dict = defaultdict(list)
for i in range(1, len(raw_ingredients)):
    flavor_dict[raw_ingredients[i-1][1].strip()] = raw_ingredients[i][0].strip()

This is the compound - ingredient mapping I've been striving for

In [665]:
flavor_dict

In [671]:
pd.Series(flavor_dict.values()).value_counts()
remove_terms = ['Not reported found in nature','n/a','Reported found in nature', 'Reported not found in nature']

In [677]:
fd = {k:v for k,v in flavor_dict.iteritems() if v not in remove_terms}

In [679]:
len(fd.keys())